All MCS0030 (SDF-based) observations will have associated metadata in the form of a metadata tarball.  The LWA1 Data Archive stores all of the metadata files that have been acquired so that observers can access the metadata.  This tutorial demonstrates the LSL interfaces to the metadata and how to extract various information.

First, lets get a metadata tarball:

In [1]:
# This may take a bit...
import os
from urllib.request import urlopen
from tempfile import mkdtemp

TEMPDIR = mkdtemp()
TGZ_FILE = os.path.join(TEMPDIR, 'LK003_0113.tgz')

fh1 = urlopen('https://lda10g.alliance.unm.edu/metadata/lwa1/observation/130921_done/LK003_0113.tgz')
fh2 = open(TGZ_FILE, 'wb')
fh2.write(fh1.read())
fh1.close()
fh2.close()
    
print("Metadata Size: %.1f kB" % (os.path.getsize(TGZ_FILE)/1024.))

Metadata Size: 470.6 kB


The lsl.common.metabundle module provides access to the contents of the tarball.  To get the SDF included in the tarball run:

In [2]:
from lsl.common import metabundle 

project = metabundle.get_sdf(TGZ_FILE)
print("Project ID: %s" % project.id)
print("Session ID: %i" % project.sessions[0].id)

Project ID: LK003
Session ID: 113


The "project" variable contains all of the observations in the SDF as well:

In [3]:
from datetime import datetime, timedelta

nObs = len(project.sessions[0].observations)
for i in range(nObs):
    currDur = project.sessions[0].observations[i].dur
    currDur = timedelta(seconds=int(currDur/1000), microseconds=(currDur*1000) % 1000000)
    
    print("  Observation #%i" % (i+1,))
    print("   Target: %s" % project.sessions[0].observations[i].target)
    print("   Mode: %s" % project.sessions[0].observations[i].mode)
    print("   Start:")
    print("    MJD: %i" % project.sessions[0].observations[i].mjd)
    print("    MPM: %i" % project.sessions[0].observations[i].mpm)
    print("   Duration: %s" % currDur)

  Observation #1
   Target: M31
   Mode: TRK_RADEC
   Start:
    MJD: 56556
    MPM: 13692273
   Duration: 4:00:00


Working with MJD and MPM values isn't particularlly transparent but a MJD/MPM pair can be easily converted to a Python datetime object storing UTC time:

In [4]:
from lsl.astro import utcjd_to_unix, MJD_OFFSET

mjd = project.sessions[0].observations[0].mjd
mpm = project.sessions[0].observations[0].mpm
tStart = utcjd_to_unix(mjd + MJD_OFFSET)
tStart += mpm / 1000.0

print("MJD: %i" % mjd)
print("MPM: %i" % mpm)
print("-> %s" % datetime.utcfromtimestamp(tStart))

MJD: 56556
MPM: 13692273
-> 2013-09-21 03:48:12.273000


The metadata tarball also stores information about the filename and DRSU barcode that the data were recorded on though the getSessionMetadata function:

In [5]:
fileInfo = metabundle.get_session_metadata(TGZ_FILE)
for obsID in fileInfo.keys():
    print(" Obs. #%i: %s on %s" % (obsID, fileInfo[obsID]['tag'], fileInfo[obsID]['barcode']))

 Obs. #1: 056556_000068631 on S15TCV13S0001


Finally, the metadata also contains the ASP settings before and after the observation so that observers can find out what ASP was set to during their observations.  If you have the dbm.gnu Python module you can access this information through the getASPConfigurationSummary function:

In [6]:
try:
    # Make sure we can read GDBM files
    from dbm import gnu
    
    for which in ('Beginning', 'End'):
        aspConfig = metabundle.get_asp_configuration_summary(TGZ_FILE, which=which)
        print('  %s' % which)
        for k in aspConfig.keys():
            print('    %s: %i' % (k, aspConfig[k]))
except ImportError:
    print('Cannot load the dbm.gnu module')

  Beginning
    filter: 0
    at1: 8
    at2: 6
    atsplit: 15
  End
    filter: 0
    at1: 8
    at2: 6
    atsplit: 15
